# Reward Graph Explained

### What is the Reward Graph?


After starting your training job, you can examine the training metrics of rewards and track completion per episode to ascertain the training job's performance of your model. On the AWS DeepRacer console, the metrics are displayed in the Reward graph

<img src="images/Reward1.PNG">

There are three metrics which are displayed on the Reward Graph:
1. Averaged reward per iteration

2. Progress per episode

3. Averaged progress per iteration

You can choose to view the reward gained per episode, the averaged reward per iteration, the progress per episode, the averaged progress per iteration or any combination of them. 

To do so, toggle the Reward (Episode, Average) or Progress (Episode, Average) switches at the bottom of Reward graph. The reward and progress per episode are displayed as scattered plots in different colors. The averaged reward and track completion are displayed by line plots and start after the first iteration.

The range of rewards is shown on the left side of the graph and the range of progress (0-100) is on the right side. To read the exact value of of a training metric, move the mouse near to the data point on the graph.

A training job is good if the averaged reward and track completion show trends to converge. In particular, the model has likely converged if the progress per episode continuously reach 100% and the reward levels out. If not, clone the model and retrain it.

## How is the Reward graph populated

* AWS DeepRacer leverages __Amazon SageMaker__ to train your model behind the scenes and uses __AWS RoboMaker__ to simulate the agent's interaction with the environment.

* Model iteration picker enables your AWS DeepRacer simulation to cycle between training and checkpoint evaluations until the end of your training job to save you time. Now, when you initiate a clone, evaluation, or leaderboard submission, AWS looks at the deepracer_checkpoints.json file(Which is generated on the backend) and fetchs the checkpoint with the highest average progress and automatically select it for you. 

Here’s how it works:

* You start a training job.

* Robomaker’s rollout worker fetches the last version of your model (or the initial version if this is the first iteration). 

* The rollout worker collects the required episodes from the simulation environment.

* SageMaker initiates a model training iteration.

* While SageMaker is training the new iteration of your model, RoboMaker takes the current model iteration and uses it to run evaluations. RoboMaker then calculates the average progress for these evaluations and stores them in the deepracer_checkpoints.json (backend)file.

* When the SageMaker model training iteration is complete, RoboMaker uses it to collect episodes for training and steps two through six repeat.

* The cycle of training and evaluation continues until time runs out, at which point SageMaker saves your model and the json file.

__Iteration hyperparameter__ the default number of episodes between training iterations is 20. To change the number of episodes in an iteration, change Number of experience episodes between each policy-updating iteration in the Hyperparameter section when you train a model in the console: Models>Create model>Hyperparameters container (collapsed by default). Specify at least five episodes.

<img src="images/Reward3.PNG">

__Best model line__ allows you to see which of your model iterations had the highest average progress during the evaluation. The checkpoint for this iteration will be stored. A checkpoint is a snapshot of a model that is captured after each training (policy-updating) iteration.

__Reward primary y-axis__ shows the reward earned during a training iteration. To read the exact value of a reward, hover your mouse over the data point on the graph.

__Percentage track completion secondary y-axis__ shows you the percentage of the track the agent completed during a training iteration.

__Iteration x-axis__ the number of iterations completed during your training job.

__Average reward__ the average reward the agent earns during a training iteration. The average is calculated by averaging the reward earned across all episodes in the training iteration. An episode, or run, begins at the starting line and ends when the agent completes one loop around the track or at the place the vehicle left the track or collided with an object. Toggle the switch to hide this data.

__Average percentage completion (training)__ is the average percentage of the track completed by the agent in all training episodes in the current training iteration. To hide Average percentage completion (training) data, toggle the switch.

__Average percentage completion (evaluating)__ is the average percentage of the track completed by the agent in all episodes run during the evaluation period, while SageMaker is updating the model, using the current iteration of your model. To hide Average percentage completion (evaluating) data, toggle the switch.

Refresh button manually updates the metric display. The graph is automatically updated every 10 seconds

### How to use the Reward Graph

It is essential to use the Reward graph as a good indicator for
* Training time of model
* Robustness of training model

#### How long one should train their models

A good indictor of how long one should train the model or stop the training can be deciphered by watching the graph when it has likely converged if the progress per episode continuously reach 100% and the reward levels out.

This can be seen in the example below

<img src="images/Reward2.PNG">

After about 400 iterations all the three metrics do not improve significantly hence it would be a good idea to either stop training the model post 600 iterations after looking at the metrics not improving and moving around the same values it had achieved previously 

Another reason to stop training the model post it achieves 100% track completion is because the continuing training will cause overfitting in the model. An over-trained model won't perform well in the real world or new tracks because it can't handle even minor variations between the simulated track and the real environment.

#### How Robust the training model is

Typically the reward graph metrics should rise in value initially and then plateau later. When the values of the metrics are near to previous values after plateauing shows that the model has learnt in well defined way and would preform consistently on mutiple runs during the model testing phase. In case the values tend to fluctuate drastically or never reach a plateau shape it then shows that the model needs more training time or is very volatile and requires either a change in the Reward function or tuning of hyperparameters